##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

In [1]:
!pip install pip==21.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pip-tools 6.13.0 requires pip>=22.2, but you have pip 21.3.1 which is incompatible.


In [2]:
import cv2

In [3]:
!unzip -qq label_images.zip

In [4]:
IMAGES_PATH = 'label_images'

## Making a New Model



In [5]:
!pip install -q mediapipe-model-maker

     |████████████████████████████████| 127 kB 3.1 MB/s            
     |████████████████████████████████| 611 kB 6.5 MB/s            
     |████████████████████████████████| 35.6 MB 488 kB/s            
     |████████████████████████████████| 2.7 MB 78.4 MB/s            
     |████████████████████████████████| 5.2 MB 74.5 MB/s            
     |████████████████████████████████| 43 kB 2.2 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 106 kB 66.6 MB/s            
     |████████████████████████████████| 1.7 MB 52.8 MB/s            
     |████████████████████████████████| 242 kB 68.8 MB/s            
     |████████████████████████████████| 589.8 MB 13 kB/s             
     |████████████████████████████████| 2.7 MB 63.1 MB/s            
     |████████████████████████████████| 5.2 MB 69.6 MB/s            


In [ ]:
from mediapipe_model_maker import gesture_recognizer

In [7]:
# Load the rock-paper-scissor image archive.
data = gesture_recognizer.Dataset.from_folder(
    dirname=IMAGES_PATH,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)

# Split the archive into training, validation and test dataset.
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

In [8]:
# Train the model
hparams = gesture_recognizer.HParams(export_dir="suhwa_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 8)                 1032      
 out (Dense)                                                     
                                                             

In [9]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

5/5 [==============================] - 0s 6ms/step - loss: 0.5959 - categorical_accuracy: 0.6000
Test loss:0.5958923697471619, Test accuracy:0.6000000238418579


In [11]:
# Export the model bundle.
model.export_model()

# Rename the file to be more descriptive.
!mv suhwa_model/gesture_recognizer.task suhwas.task

Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/canned_gesture_classifier.tflite


In [ ]:
img = cv2.imread("test.jpg")
cv2_imshow(img)

In [14]:
# Imports neccessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os

# Create a GestureRecognizer object.
model_path = os.path.abspath("suhwas.task")
recognizer = vision.GestureRecognizer.create_from_model_path(model_path)

# Load the input image.
image = mp.Image.create_from_file('test.jpg')

# Run gesture recognition.
recognition_result = recognizer.recognize(image)

# Display the most likely gesture.
top_gesture = recognition_result.gestures[0][0]
print(f"Gesture recognized: {top_gesture.category_name} ({top_gesture.score})")

Gesture recognized:  (0.5353529453277588)
